In [2]:
!pip install datasets
!pip install transformers
!pip install accelerate
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cpu


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
There is no specific figure for how many helicopters a human can eat in one sitting. However, according to some reports, humans can have a variety of foods, including fruits, vegetables, protein-rich foods, and whole grains, without getting too full. In general, humans tend to eat moderately and take breaks to refuel their body.


In [ ]:
import time
import torch
from transformers import pipeline
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
system_message = {"role": "system", "content": ("You are a Premium Chatbot who thinks before speaking and uses 2 paragraphs: one for Reasoning and another for Response. When replying, format your answer using the following tags exactly: output your reasoning with '<|assistant reasoning|>' followed by your chain-of-thought explanation, and then output your final reply with '<|assistant response|>' followed by your answer. Also, please use lowercase for '<|system|>' and '<|user|>' tags.")}
conversation = [system_message]
def reformat_output(output_text: str) -> str:
    reformatted = output_text.replace("<System>", "<|system|>").replace("<User>", "<|user|>")
    if "<assistant>" in reformatted:
        before, assistant_block = reformatted.split("<assistant>", 1)
        if "Reasoning:" in assistant_block and "Answer:" in assistant_block:
            reasoning_part = assistant_block.split("Answer:")[0]
            response_part = assistant_block.split("Answer:")[1]
            reasoning_part = reasoning_part.replace("Reasoning:", "").strip()
            response_part = response_part.strip()
            new_assistant_text = f"<|assistant reasoning|> {reasoning_part}\n<|assistant response|> {response_part}"
            reformatted = before + new_assistant_text
        else:
            reformatted = reformatted.replace("<assistant>", "<|assistant reasoning|>")
    return reformatted
print("Type 'bye' to exit the chat.")
while True:
    user_input = input("User: ")
    if user_input.strip().lower() == "bye":
        print("Chatbot: Farewell, Sir!")
        break
    conversation.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
    start_time = time.time()
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    end_time = time.time()
    response_time = end_time - start_time
    generated_text = outputs[0]["generated_text"]
    formatted_output = reformat_output(generated_text)
    print("Chatbot:")
    print(formatted_output)
    print(f"\nResponse Time: {response_time:.2f} seconds")
    print("\n" + "-"*50 + "\n")
    conversation.append({"role": "assistant", "content": generated_text})


Device set to use cpu


Type 'bye' to exit the chat.
User: how many "r" in "strawberry"?
Chatbot:
<|system|>
You are a Premium Chatbot who thinks before speaking and uses 2 paragraphs: one for Reasoning and another for Response. When replying, format your answer using the following tags exactly: output your reasoning with '<|assistant reasoning|>' followed by your chain-of-thought explanation, and then output your final reply with '<|assistant response|>' followed by your answer. Also, please use lowercase for '<|system|>' and '<|user|>' tags.</s>
<|user|>
how many "r" in "strawberry"?</s>
<|assistant|>
There are four "r" in the word "strawberry": "st", "ra", "b", and "f".

Response Time: 62.79 seconds

--------------------------------------------------



In [3]:
import time
import torch
import gradio as gr
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
system_message = {"role": "system", "content": ("You are a Premium Chatbot who thinks before speaking and uses 2 paragraphs: one for Reasoning and another for Response. When replying, format your answer using the following tags exactly: output your reasoning with '<|assistant reasoning|>' followed by your chain-of-thought explanation, and then output your final reply with '<|assistant response|>' followed by your answer. Also, please use lowercase for '<|system|>' and '<|user|>' tags.")}
conversation = [system_message]

def reformat_output(output_text: str) -> str:
    reformatted = output_text.replace("<System>", "<|system|>").replace("<User>", "<|user|>")
    if "<assistant>" in reformatted:
        before, assistant_block = reformatted.split("<assistant>", 1)
        if "Reasoning:" in assistant_block and "Answer:" in assistant_block:
            reasoning_part = assistant_block.split("Answer:")[0]
            response_part = assistant_block.split("Answer:")[1]
            reasoning_part = reasoning_part.replace("Reasoning:", "").strip()
            response_part = response_part.strip()
            new_assistant_text = f"<|assistant reasoning|> {reasoning_part}\n<|assistant response|> {response_part}"
            reformatted = before + new_assistant_text
        else:
            reformatted = reformatted.replace("<assistant>", "<|assistant reasoning|>")
    return reformatted

def chat(user_input, state):
    if user_input.strip().lower() == "bye":
        return "Chatbot: Farewell, Sir!", state
    state.append({"role": "user", "content": user_input})
    prompt = pipe.tokenizer.apply_chat_template(state, tokenize=False, add_generation_prompt=True)
    start_time = time.time()
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    end_time = time.time()
    response_time = end_time - start_time
    generated_text = outputs[0]["generated_text"]
    formatted_output = reformat_output(generated_text)
    state.append({"role": "assistant", "content": generated_text})
    result = f"{formatted_output}\n\nResponse Time: {response_time:.2f} seconds"
    return result, state

with gr.Blocks() as demo:
    gr.Markdown("## LogicLink Chatbot")
    chatbot_state = gr.State(value=conversation)
    chatbot_output = gr.Textbox(label="Chatbot Response", interactive=False)
    user_input_text = gr.Textbox(placeholder="Type your message here...", label="Your Message")
    send_btn = gr.Button("Send")
    send_btn.click(chat, inputs=[user_input_text, chatbot_state], outputs=[chatbot_output, chatbot_state])

demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7dbc0f57166ec7ec21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
def generate_formatted_output(user_prompt):
    system_text = (
        "You are a Premium Chatbot who thinks before speaking and always responds "
        "in the specified format."
    )

    # Reasoning is based on the provided reasoning prompt
    assistant_reasoning = "Different ways to approach the Query : " + user_prompt + " [Insert chain-of-thought reasoning here.]"

    # Response is based on the provided response prompt (echoing the query here)
    assistant_response = user_prompt  # Replace with your actual response logic if needed

    formatted_output = (
        f"<|system|>\n{system_text}\n"
        f"<|user|>\n{user_prompt}\n"
        f"<|assistant reasoning|>\n{assistant_reasoning}\n"
        f"<|assistant response|>\n{assistant_response}"
    )

    return formatted_output

# Example usage:
user_input = "hello how are you"
output = generate_formatted_output(user_input)
print(output)


<|system|>
You are a Premium Chatbot who thinks before speaking and always responds in the specified format.
<|user|>
hello how are you
<|assistant reasoning|>
Different ways to approach the Query : hello how are you [Insert chain-of-thought reasoning here.]
<|assistant response|>
hello how are you


In [9]:
import gradio as gr
import time
import torch
from transformers import pipeline

# Initialize the text-generation pipeline with your model
pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

def chat(user_query):
    # Construct the prompts for reasoning and final response.
    reasoning_prompt = "Reflect on all potential nuances, underlying concepts, and alternative perspectives before forming a final answer for the following query : " + user_query
    response_prompt = user_query

    start_time = time.time()
    # Generate reasoning output
    reasoning_out = pipe(reasoning_prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    reasoning_output = reasoning_out[0]["generated_text"]

    # Generate final response output
    response_out = pipe(response_prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    response_output = response_out[0]["generated_text"]

    end_time = time.time()
    total_time = end_time - start_time

    # Format the output according to the specified tags.
    formatted = (
        f"<|User|> {user_query}\n\n"
        f"<|Assistant Reasoning|>\n {reasoning_output}\n\n\n"
        f"<|Assistant Response|>\n {response_output}\n\n\n"
        f"Response Time: {total_time:.2f} seconds"
    )
    return formatted

with gr.Blocks() as demo:
    gr.Markdown("## LogicLink Chatbot")
    user_input_text = gr.Textbox(label="Your Query", placeholder="Type your query here...", lines=2)
    chatbot_output = gr.Textbox(label="Chatbot Output", interactive=False)
    send_btn = gr.Button("Send")
    send_btn.click(chat, inputs=user_input_text, outputs=chatbot_output)

demo.launch()


Device set to use cuda:0


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48a6f79fd56b5114a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
